In [3]:
import os
import sys
import optparse
import random
from itertools import  cycle
import numpy as np
from agent.agent import Agent
from algos.ppo import *
import time

# we need to import python modules from the $SUMO_HOME/tools directory
# i adden two hidden layers in ppo, if you have an error you know wht to do


In [4]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/NEW')

his = []
state_dim = 12
action_dim = 2
agent = Agent(act_dim=action_dim,state_dim=state_dim)

render = False
log_interval = 10           # print avg reward in the interval
max_episodes = 600       # max training episodes
max_timesteps = 3600         # max timesteps in one episode
n_latent_var = 20           # number of variables in hidden layer
update_timestep = 50000      # update policy every n timesteps
lr = 0.002
betas = (0.9, 0.999)
gamma = 0.99                # discount factor
K_epochs = 8               # update policy for K epochs
eps_clip = 0.6          # clip parameter for PPO
random_seed = 47
    #############################################
name = f'50n_2hidden_layersv3_{time.time()}'
if random_seed:
    torch.manual_seed(random_seed)
        #env.seed(random_seed)
    
memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr,betas)
    
    # logging variables
running_reward = 0
avg_length = 0
timestep = 0
min_objective = float('inf')
rewards = []
    # training loop
print(ppo.get_lr())
for i_episode in range(1, max_episodes+1):
    rewards = []
    agent.terminate = False
    
    agent.reset()
    state = agent.get_state()
    #print(state)
    agent.step()
    
    for t in range(max_timesteps):
        agent.step()
        timestep += 5
            # Running policy_old:
        action = ppo.policy_old.act(state, memory)
        #print(action)
        state, reward, done = agent.action(action)
        rewards.append(reward)
            # Saving reward and is_terminal:
        #print(state, reward, done)
        #print('reward ' + str(reward))
        #if done:print('iiiiiiiiiiiiiiiii')
        memory.rewards.append(reward)
        memory.is_terminals.append(done)
                    # update if its time
        if timestep % update_timestep == 0:
            #print('yes')
            #print(reward, action)
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0
        
            #if render:
             #   env.render()
        if done:
            #agent.reset()
            break
        running_reward += reward
    his.append(agent.waiting_time)
    #print(sum(rewards)/len(rewards))
    writer.add_scalar("waiting_time/train", agent.his[-1][0], i_episode)
    writer.add_scalar("CO2", agent.his[-1][1], i_episode)
    writer.add_scalar("rewards", sum(rewards)/len(rewards), i_episode)
    #print(agent.waiting_time)
    avg_length += t
    
        # stop training if avg_reward > solved_reward
    if min_objective > agent.his[-1][0]:
        print("########## Solved! ##########")
        min_objective = agent.his[-1][0]
        print(min_objective)
        torch.save(ppo.policy.state_dict(), f'models_save/PPO_{name}.pth')
       # break
            
        # logging
    if i_episode % log_interval == 0:
        avg_length = int(avg_length/log_interval)
        running_reward = int((running_reward/log_interval))
            
        print(f'Episode {i_episode} \t avg length: {avg_length} \t reward: {running_reward}')
        running_reward = 0
        avg_length = 0
writer.flush()
writer.close()

            

yes
0.002 (0.9, 0.999)
0.002
0.002
907200.0
########## Solved! ##########
907200.0
597554.0
########## Solved! ##########
597554.0
784170.0
715699.0
690482.0
715021.0
660754.0
658777.0
661068.0
810796.0
Episode 10 	 avg length: 556 	 reward: -74182
844566.0
668435.0
646128.0
628159.0
656766.0
726751.0
786360.0
728475.0
946625.0
882135.0
Episode 20 	 avg length: 555 	 reward: -75236
826177.0
861470.0
1010683.0
731102.0
1113575.0
972400.0
814999.0
1013980.0
750912.0
984171.0
Episode 30 	 avg length: 559 	 reward: -75253
768934.0
923379.0
865616.0
808011.0
737108.0
886268.0
1031701.0
1042509.0
1288053.0
1113342.0
Episode 40 	 avg length: 557 	 reward: -77784
1141902.0
949996.0
1063288.0
1060807.0
1050611.0
1438990.0
1046475.0
1027998.0
915683.0
1219164.0
Episode 50 	 avg length: 566 	 reward: -80024
1080375.0
1218905.0
918535.0
1060537.0
782532.0
705844.0
713198.0
852922.0
587145.0
########## Solved! ##########
587145.0
878716.0
Episode 60 	 avg length: 559 	 reward: -74267
778830.0
75152

In [ ]:
writer = SummaryWriter(r'out\here')

writer.log_dir

In [3]:
#import matplotlib
import matplotlib.pyplot as plt
#import numpy as np
#print(agent.his)
y = range(len(agent.his[::10]))
fig, ax = plt.subplots()
ax.plot(y, agent.his[::10])

plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
import pandas as pd

df = pd.DataFrame(np.array(agent.his))
df.to_csv('historyv2.csv',)

In [ ]:
num_inputs  = state_dim
num_outputs = action_dim

#Hyper params:
hidden_size      = 15
lr               = 3e-4
num_steps        = 20
mini_batch_size  = 5
ppo_epochs       = 4
threshold_reward = -200

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

max_frames = 15000
frame_idx  = 0
test_rewards = []
agent = Agent(act_dim=2,state_dim=9)


In [ ]:
agent.reset()
state = agent.get_state()
early_stop = False

while frame_idx < max_frames and not early_stop:
    agent.reset()

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        
        next_state, reward, done, _ = agent.action(action.cpu().numpy())
        agent.step()
        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            plot(frame_idx, test_rewards)
            if test_reward > threshold_reward: early_stop = True
            

    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

In [9]:
a  = torch.tensor([1,2,3,np.nan])
#torch.zeros_like(a)
torch.where(torch.isnan(a), torch.zeros_like(a), a)
from torch.utils.tensorboard import SummaryWriter

In [8]:
!pip install tensorboard

  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.1.0
    Uninstalling setuptools-39.1.0:
      Successfully uninstalled setuptools-39.1.0
  Attempting uninstall: requests
    Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
twisted 20.3.0 requires attrs>=19.2.0, but you have attrs 18.1.0 which is incompatible.
You should consider upgrading via the 'e:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import traci.constants as tc
first = False
def run():
    """execute the TraCI control loop"""
    step = 0
    test = 0
    s = 0
    #network.write("data/cross.net.xml")
    #print(type(traci.trafficlight.getCompleteRedYellowGreenDefinition('0')[0].getPhases()[0]))
    lst = traci.trafficlight.getIDList()
    #print(lst)
    #traci.trafficlight.getPhase()
    
    for i in lst:
        logic = traci.trafficlight.getAllProgramLogics(i)
        print(logic)
        
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        test+= 1
        #print(traci.vehicle.getAccumulatedWaitingTime)
        for v in traci.vehicle.getIDList():
            s+=traci.vehicle.getWaitingTime(v)
            #traci.simulation.del
        step += 1
        lane_lst = traci.lanearea.getIDList()
        #print(lane_lst)
        act = 1
        if step%50==0:
            waiting_time,carbon_omission = agent.get_reward(traci)
            print(agent.get_reward(traci))
            print(agent.get_state(traci))
            state = agent.get_state(traci)
            i = 0
            for info in state:
                if info[0] > 10:
                    print('changed')
                    act = 0


        
        #if step % 20 == 0 :
         #   act = 0
        agent.action(act,traci)
        if step >=200:
            break
    print(s)    
    traci.close()
    sys.stdout.flush()


def get_options():
    optParser = optparse.OptionParser()
    optParser.add_option("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
    options, args = optParser.parse_args()
    return options


In [5]:
#options = get_options()
var = 0
label = 0
generate_routefile()
done = False
while not done:
    try:
        traci.start(['sumo-gui', "-c", "data/new/cross.sumocfg",
        "--tripinfo-output", "tripinfo.xml"],label=str(label))
        done = True
    except:
        label+=1

run()


(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=25.0, state='GrGr', minDur=25.0, maxDur=25.0, next=()), Phase(duration=20.0, state='yryr', minDur=20.0, maxDur=20.0, next=()), Phase(duration=42.0, state='rGrG', minDur=42.0, maxDur=42.0, next=()), Phase(duration=24.0, state='ryry', minDur=24.0, maxDur=24.0, next=())), subParameter={}),)
(0.0, 23248.176340314367)
[(3, 11.109450591881947), (2, 16.204270759607173), (0, -1.0), (0, 10.869061315818856)]
(262.0, 45949.49188311631)
[(7, 4.782935074153847), (5, 3.296839882560074), (0, -1.0), (0, -1.0)]
(950.0, 68307.8137217103)
[(11, 1.8405738498513542), (9, 0.0), (0, -1.0), (0, -1.0)]
changed


FatalTraCIError: connection closed by SUMO

In [ ]:
model = ppo()
agent = Agent()
max_iter = 1000
while i < max_iter and not early_stop:

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, done, _ = agent.act(action.cpu().numpy())

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        i += 1
        
        if i % 1000 == 0:
            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            if test_reward > threshold_reward: early_stop = True
            

    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)



In [11]:
traci.lanearea.getIDList()

('e2det_1i_0', 'e2det_2i_0', 'e2det_3i_0', 'e2det_4i_0')

In [18]:
cy = cycle((1,2,3))

In [3]:
a = [0]
a[-1]
with open("data/cross.rou.xml", "w") as routes:
    print(a)

[0]


In [17]:

!pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html



Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.7.1%2Bcpu-cp36-cp36m-win_amd64.whl (184.2 MB)
  Using cached https://download.pytorch.org/whl/torchaudio-0.7.2-cp36-none-win_amd64.whl (103 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.8.2%2Bcpu-cp36-cp36m-win_amd64.whl (805 kB)
You should consider upgrading via the 'e:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
range(8)

range(0, 8)

In [10]:
!source activate e:/envirments/ml/Scripts/Activate.ps1

'source' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
!source activate source activate e:/envirments/ml/Scripts/python
!python -m ipykernel install --user --name e:/envirments/ml/Scripts/Activate.ps1 --display-name "Python (myenv)"


'source' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "E:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "E:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "E:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 480, in start
    return self.subapp.start()
  File "E:\Anaconda3\lib\site-packages\ipykernel\kernelspec.py", line 176, in start
    prefix=opts.prefix, display_name=opts.display_name)
  File "E:\Anaconda3\lib\site-packages\ipykernel\kernelspec.py", line 133, in install
    path, kernel_name=kernel_name, user=user, prefix=prefix)
  File "E:\Anaconda3\lib\site-packages\jupyter_client\ker

In [20]:
#traci.start()
import time
time.localtime()

time.struct_time(tm_year=2021, tm_mon=2, tm_mday=25, tm_hour=16, tm_min=1, tm_sec=17, tm_wday=3, tm_yday=56, tm_isdst=0)